# SEMESO 2023 - Mini Curso - Data Science no Mundo das Criptomoedas: da Blockchain ao MLOps 

Felipe Lana Machado

## Etapas:

1. Obtenção dos dados da moeda Illuvium (ILV) na rede ethereum.
    - Utilização da lib web3.py.
    - Utilização da API Alchemy. 
2. Pre-processamento dos dados obtidos.
3. Desenvolvimento de features dentro dos dados obtidos.
4. CI/CD com github actions.
5. Princípios do MLOps.

In [18]:
import warnings
import json
import os
from typing import List, Dict
from urllib3.exceptions import NotOpenSSLWarning
warnings.simplefilter('ignore', NotOpenSSLWarning)
from src.blockchain.get_ethereum_data import Blockchain
from src.preprocessing.illuvium_transactions_processing import IlluviumTransactionsProcessing
from src.utils.project_paths import DATA_RAW, DATA_PROCESSED

## Obtenção dos dados

In [19]:
blockchain = Blockchain(abi='illuvium')

In [2]:
illuvium_started = 12084123
data = blockchain._get_events(start_block=illuvium_started)

list_of_files = os.listdir(DATA_RAW)
size = len([file for file in list_of_files if f'transfer_data_' in file])
with open(os.path.join(DATA_RAW, f'transfer_data_{size+1}.json'), 'w') as json_file:
    json.dump(data, json_file, indent=4)

100%|██████████| 614/614 [11:30<00:00,  1.12s/it]


In [20]:
size = 1

In [21]:
def load_json(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"{filename} not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {filename}.")
        return None

data = load_json(os.path.join(DATA_RAW, f"transfer_data_{size+1}.json"))

## Pre-Processamento

In [22]:
transactions_processing = IlluviumTransactionsProcessing(blockchain=blockchain)

In [23]:
data = transactions_processing.remove_duplicates_by_field(field="txhash", data=data)
len(data)

502626

In [24]:
data = transactions_processing.create_ilv_value_field(data=data)
len(data)

502626

In [25]:
data[-1]

{'txhash': '0x4b54730b5cfeb4ac62ed5d3f6f91e0fb89b82ce4ae1514adad44af23bc6b6b12',
 'blockNumber': 18219663,
 'from': '0x9993e1fB39CA76cB537e6a8eD460AB19351bd419',
 'to': '0x75e89d5979E4f6Fba9F97c104c2F0AFB3F1dcB88',
 'value': 4969214249502901718,
 'value_ilv': Decimal('4.969214249502901718')}

In [28]:
list_of_files = os.listdir(DATA_PROCESSED)
size = len([file for file in list_of_files if f'transfer_data_processed_' in file])
with open(os.path.join(DATA_PROCESSED, f'transfer_data_processed_{size}.json'), 'w') as json_file:
    json.dump(data, json_file, indent=4, default=str)

## Desenvolvimento de Features

### Total de Moedas Transferidas por Bloco

#### - block_total_transfered_strategy

Soma dos valores das transações para um bloco específico.


### Número de Transações por Bloco

#### - block_transactions_count_strategy

Quantidade de transações que ocorrem em um bloco específico.

### Transações de Alto Valor (Whales)

#### - high_value_transactions_strategy

Número ou proporção de transações em um bloco que envolvem uma quantidade significativamente grande de moedas.

Para esta feature, consideraremos uma transação de alto valor (whale) aquela que tem um valor acima de um certo limite. Por exemplo, se dissermos que uma transação de 1.000 unidades da moeda é considerada de alto valor, então quaisquer transações com 1.000 unidades ou mais serão classificadas como transações "whale".


### Distribuição de Tamanho de Transações:

#### - transactions_distribuition_size_strategy

Histograma ou distribuição percentual das transações por tamanho em um bloco específico.

Para essa feature, queremos ter uma ideia da distribuição das transações em categorias de tamanho. Por exemplo, podemos ter categorias como "pequeno", "médio" e "grande" e contar quantas transações se encaixam em cada uma dessas categorias.

Vamos supor os seguintes limites para as categorias:

Pequeno: <=500
Médio: >500 e <=1500
Grande: >1500


### Carteiras Ativas por Bloco:

#### - active_wallets_by_block_strategy

Número de carteiras únicas que participaram de transações em um bloco específico.

### Novas Carteiras por Bloco:

#### - new_wallets_by_block_strategy

Quantidade de carteiras que aparecem pela primeira vez em um bloco específico, indicando novos participantes entrando no ecossistema.

### Carteiras de Alto Volume (Top Holders):

#### high_volume_wallets_strategy

Com base em 'from' e 'to', identifique as carteiras que, ao longo de vários blocos, mantêm ou movimentam os maiores volumes da moeda.

### Relação Valor/Bloco:

#### - ratio_value_block_strategy

Uma medida que compara o valor transferido em relação ao número do bloco. Pode indicar se o valor transferido está aumentando ou diminuindo ao longo do tempo.

### Transações Recorrentes entre Carteiras:

#### - Recurrent_wallet_transactions_strategy

Identifique pares de carteiras ('from' e 'to') que têm transações recorrentes ao longo de vários blocos.

### Concentração de Valor:

#### value_concentration_strategy

Calcule a proporção do valor total transferido que é movimentado pelas top N carteiras (por exemplo, top 100) em vários blocos. Isso pode indicar se a moeda está concentrada entre poucos titulares ou distribuída mais amplamente.

## CI / CD - Github Actions

## MLOPs

In [1]:
import os

def create_test_files(src_directory, dest_directory):
    """
    Cria arquivos de teste em dest_directory baseado nos arquivos de src_directory.

    Args:
        src_directory (str): O caminho da pasta de origem.
        dest_directory (str): O caminho da pasta de destino.
    """

    # Se a pasta de destino não existir, crie-a
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)

    # Listar todos os arquivos na pasta de origem
    for filename in os.listdir(src_directory):
        # Ignora subdiretórios
        if os.path.isfile(os.path.join(src_directory, filename)):
            # Divide o nome do arquivo e a extensão
            base, ext = os.path.splitext(filename)
            # Cria o nome do novo arquivo com "_test" antes da extensão
            new_filename = f"{base}_test{ext}"
            new_filepath = os.path.join(dest_directory, new_filename)

            # Cria o arquivo na pasta de destino (ele estará vazio)
            with open(new_filepath, 'w') as f:
                pass

# Exemplo de uso:
create_test_files('src/features', 'tests/features')
